# PARALLELIZE RANDOM FOREST

### Hàm hỗ trợ

In [1]:
import numpy as np
import pandas as pd
import numba as nb
import random
import time
from numba import prange, njit, jit, typed, types
from numba.typed import List, Dict
from numba.core import types

# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Hàm hỗ trợ phân tách tập train - test

def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

# Hàm hỗ trợ lọc và đến các giá trị unique

@nb.jit
def numba_unique_with_counts(arr):

    unique_values = np.empty(0)
    value_counts = np.empty(0)

    for value in arr:
        if value not in unique_values:
            np.append(unique_values, value)
            np.append(value_counts, 1)
        else:
            index = unique_values.index(value)
            value_counts[index] += 1

    return unique_values, value_counts

### Decision Tree


In [4]:
'''
Kiểm tra data có thuộc một label duy nhất
'''
@nb.jit
def checkPurity(data):
    uniqueVals, _ = numba_unique_with_counts(data[:, -1])
    if len(uniqueVals) == 1:
        return True
    else:
        return False


'''
Xác định label phổ biến nhất còn trong data và trả về label đó

'''
@nb.jit
def classifyData(data):
    uniqueClasses, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)

    # print(uniqueClassesCounts)

    return uniqueClasses[uniqueClassesCounts.argmax()]


'''
Xác định cột (thuộc tính) và các giá trị split tiềm năng của cột đó
'''
# def getPotentialSplits(data):
#     potentialSplits = {}
#     _, columns = data.shape

#     # Lấy hết thuộc tính (trừ cột label cuối cùng) gán vào columnsIndices (List gồm index của các thuộc tính)
#     columnsIndices = list(range(columns - 1))

#     for column in columnsIndices:
#         values = data[:, column]
#         # Xét độ unique của mỗi thuộc tính
#         uniqueValues = np.unique(values)

#         # Nếu thuộc tính chỉ có đúng 1 giá trị thì potentialSplits tại cột (thuộc tính) đó = chính giá trị đó
#         if len(uniqueValues) == 1:
#             potentialSplits[column] = uniqueValues

#         # còn không thì potentialSplits tại cột (thuộc tính) đó sẽ chứa các giá trị trung bình giữa 2 giá trị kế nhau trong mảng uniqueValues
#         else:
#             potentialSplits[column] = []
#             for i in range(len(uniqueValues)):
#                 if i != 0:
#                     currentValue = uniqueValues[i]
#                     previousValue = uniqueValues[i - 1]
#                     potentialSplits[column].append((currentValue + previousValue) / 2)
#     return potentialSplits

@nb.jit
def getPotentialSplits(data):
  columns = data.shape[1]


  columnsIndices = np.arange(columns - 1)
  potentialSplits = np.empty(columns - 1, dtype=object)

  for col in columnsIndices:
    values = data[:, col]

    uniqueValues, _ = numba_unique_with_counts(values)

    if len(uniqueValues) == 1:
      potentialSplits[col] = uniqueValues

    else:
      # splitValues = np.empty(len(uniqueValues) - 1)

      if len(uniqueValues) > 0:
        splitValues = np.empty(len(uniqueValues) - 1)
      else:
        splitValues = np.empty(0)

      for i in range(len(uniqueValues)):
          if i != 0:
              currentValue = uniqueValues[i]
              previousValue = uniqueValues[i - 1]
              splitValues[i-1] = (currentValue + previousValue) / 2

      # Vẫn là mảng một chiều nên access vô phần tử bằng [][] còn [ , ] thì ko dc
      potentialSplits[col] = splitValues

  return potentialSplits


'''
# Tách data thành 2 phần dựa vào splitValue
'''
@nb.jit
def splitData(data, splitColumn, splitValue):
    # Lấy data của cột thuộc tính đang đc xét để so sánh
    splitColumnValues = data[:, splitColumn]

    # Tách nguyên data (không phải chỉ mỗi data ở cột splitColumnValues) thành hai phần, một phần gồm các giá trị ở cột splitColumnValues <= splitValue và ngược lại
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]

'''
# Tính entropy của từng phần data
'''

# def calculateEntropy(data):
#     # Tính entropy của data dựa vào tỷ lệ labels mà data thuộc về
#     _, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
#     probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()
#     return sum(probabilities * -np.log2(probabilities))

@nb.jit(parallel=True, cache=True)
def calculateEntropy(data):

    # Tính entropy của data dựa vào tỷ lệ labels mà data thuộc về
    _, uniqueClassesCounts = numba_unique_with_counts(data[:, -1])
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()

    s = 0.0

    for i in nb.prange(len(probabilities)):
      s += ( probabilities[i] * -np.log2(probabilities[i]) )

    return s


'''
# Tính tổng entropy
'''

@nb.jit(cache=True)
def calculateOverallEntropy(dataBelow, dataAbove):

    # print(dataBelow)
    # print(dataAbove)

    # Lấy tỷ lệ số lượng data ở nửa trên so với số lượng data
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))

    # Lấy tỷ lệ số lượng data ở nửa dưới so với số lượng data
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))

    # pDataBelow = np.array(pDataBelow)
    # pDataAbove = np.array(pDataAbove)

    return pDataBelow * calculateEntropy(np.array(dataBelow)) + pDataAbove * calculateEntropy(np.array(dataAbove))


'''
Xác định cột (thuộc tính) và giá trị tiềm năng duy nhất của cột đó (splitPoint có entropy nhỏ nhất)
'''
# def determineBestSplit(data, potentialSplits):
#     overallEntropy = 9999
#     bestSplitColumn = 0
#     bestSplitValue = 0

#     # Xét từng key là thuộc tính trong potentialSplits
#     for splitColumn in potentialSplits:
#         # Xét từng value của từng thuộc tính trong potentialSplits
#         for splitValue in potentialSplits[splitColumn]:
#             # Tách nguyên data của làm hai phần nhỏ hơn và lớn hơn so với value đang xét
#             dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
#             # Xét Entropy của 2 phần data đó, nếu nhỏ nhất thì lấy
#             currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
#             if currentOverallEntropy <= overallEntropy:
#                 overallEntropy = currentOverallEntropy
#                 bestSplitColumn = splitColumn
#                 bestSplitValue = splitValue
#     return bestSplitColumn, bestSplitValue


@nb.jit(parallel=True, cache=True)
def determineBestSplit(data, potentialSplits):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0

    # Xét từng key là thuộc tính trong potentialSplits
    for splitColumn in nb.prange(len(potentialSplits)):
        # Xét từng value của từng thuộc tính trong potentialSplits
        for splitValue in nb.prange(len(potentialSplits[splitColumn])):
            # Tách nguyên data của làm hai phần nhỏ hơn và lớn hơn so với value đang xét
            dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
            # Xét Entropy của 2 phần data đó, nếu nhỏ nhất thì lấy
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = splitColumn
                bestSplitValue = splitValue
    return bestSplitColumn, bestSplitValue


'''
Xây dựng cây
'''
@nb.jit(cache=True)
def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, COLUMN_HEADERS = None):
    # Nếu mới bắt đầu khởi tạo cây
    if currentDepth == 0:
        # global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns.to_numpy()
        data = dataFrame.to_numpy()
    else:
        data = dataFrame
        COLUMN_HEADERS = COLUMN_HEADERS
    # Xét nếu tất cả dữ liệu đều có label giống nhau || dữ liệu dữ liệu ít hơn số dữ liệu tối thiểu để xét cho một node || độ cao đã đạt tối đa
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        # Trả về label cho node cuối cùng đó
        return classifyData(data)

    # Nếu không thì tiếp tục quá trình tạo cây
    else:
        currentDepth += 1
        # Lấy ra potentialSplits là dictionaries gồm key là các thuộc tính, values là các giá trị potential của mỗi thuộc tính
        potentialSplits = getPotentialSplits(data)


        # Lấy ra cột thuộc tính và giá trị split của thuộc tính mà mang lại giá trị entropy nhỏ nhất
        splitColumn, splitValue = determineBestSplit(data, potentialSplits)

        # split data theo 2 giá trị vừa lấy đc
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)

        # Nếu tất cả data đều thuộc một label rồi thì lấy label đó cho node
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            # Đặt giá trị splitPoint cho thuộc tính đó, phần data nhỏ hơn sẽ qua yes và ngược lại
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            # Tiếp tục rẽ nhánh bằng đệ quy
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, COLUMN_HEADERS)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, COLUMN_HEADERS)
            # Nếu 2 nhánh giống nhau thì chỉ rẽ 1 nhánh, khác nhau thì rẽ 2 nhánh
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree


'''
Dùng để xét label cho tập dữ liệu bằng cách đưa mỗi cây trong randomForest
'''
@nb.jit
def classifySample(sample, decisionTree):
    # Nếu xét đến label của node lá cuối cùng thì trả về label đó
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    # So sánh với splitValue để mò đường đi tới node chứa label quyết định
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)


'''
Trả về label được dự đoán của tập data
'''
def decisionTreePredictions(dataFrame, decisionTree):
    # Dự đoán từng label của từng dòng dữ liệu
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions


'''
# So sánh độ chính xác giữa tập đang ktra và tập giá trị thật
'''
def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

### Random Forest


In [5]:
'''
Dùng kỹ thuật boostrap để lấy random một subset của data
'''
@nb.jit
def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = np.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]


'''
Xây dựng rừng cây từ các cây decision tree
'''
@nb.jit(parallel=True)
def createRandomForest(dataFrame, bootstrapSize, forestSize = 20, treeMaxDepth = 1000):
    forest = []
    total_time=0
    for i in nb.prange(forestSize):
        # Lấy random một subset của data bằng kỹ thuật boostrap
        bootstrappedDataFrame = bootstrapSample(dataFrame, bootstrapSize)
        # Xây dựng 1 cây quyết định từ subset đó
        start_time = time.time()
        decisionTree = buildDecisionTree(bootstrappedDataFrame, maxDepth = treeMaxDepth, )
        end_time = time.time()
        total_time += (end_time - start_time)
        # Thêm cây vào rừng
        forest.append(decisionTree)
    print("AVG time to build a Decision Tree =",total_time/forestSize)
    return forest


'''
Dự đoán data bằng cách đưa qua từng cây trong randomForest rồi lấy label theo số đông
'''
def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    # Đưa dữ liệu qua rừng cây (cụ thể là từng cây)
    for i in range(len(randomForest)):

        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pd.DataFrame(predictions)
    # Lấy label có số lần xuất hiện nhiều nhất của từng dòng
    return predictions.mode(axis = 1)[0]

'''
# So sánh độ chính xác giữa tập đang ktra và tập giá trị thật
'''
def calculateAccuracy(predictedResults, category):
    # So sánh label của tập dự đoán và tập real
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

### Run with car_evaluation dataset

In [6]:
dataFrame = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/random-forest-from-scratch-main/dataset_files/car_evaluation.csv")

buyingMapping = {"low": 1, "med": 2, "high": 3, "vhigh": 4}
dataFrame["buying"] = dataFrame["buying"].map(buyingMapping)

maintMapping = {"low": 1, "med": 2, "high": 3, "vhigh": 4}
dataFrame["maint"] = dataFrame["maint"].map(maintMapping)

doorsMapping = {"2": 2, "3": 3, "4": 4, "5more": 5}
dataFrame["doors"] = dataFrame["doors"].map(doorsMapping)

personsMapping = {"2": 2, "4": 4, "more": 6}
dataFrame["persons"] = dataFrame["persons"].map(personsMapping)

lugBootMapping = {"small": 1, "med": 2, "big": 3}
dataFrame["lug_boot"] = dataFrame["lug_boot"].map(lugBootMapping)

safetyMapping = {"low": 1, "med": 2, "high": 3}
dataFrame["safety"] = dataFrame["safety"].map(safetyMapping)

dataFrameTrain, dataFrameTest = trainTestSplit(dataFrame, testSize = 0.3)

print("Random Forest - Car Evaluation Dataset")
print("  Maximum bootstrap size (n) is {}".format(dataFrameTrain.shape[0]))
print("  Maximum random subspace size (d) is {}".format(dataFrameTrain.shape[1] - 1))

Random Forest - Car Evaluation Dataset
  Maximum bootstrap size (n) is 1210
  Maximum random subspace size (d) is 6


#### First time run

In [7]:
startTime = time.time()
randomForest = createRandomForest(dataFrameTrain, bootstrapSize = 1000, forestSize = 10, treeMaxDepth = 8)
buildingTime = time.time() - startTime
randomForestTestResults = randomForestPredictions(dataFrameTest, randomForest)
accuracyTest = calculateAccuracy(randomForestTestResults, dataFrameTest.iloc[:, -1]) * 100
randomForestTrainResults = randomForestPredictions(dataFrameTrain, randomForest)
accuracyTrain = calculateAccuracy(randomForestTrainResults, dataFrameTrain.iloc[:, -1]) * 100
print("  n = {}, k = {}, maxDepth = {}:".format(1000, 10, 8))
print("    accTest = {0:.2f}%, ".format(accuracyTest), end = "")
print("accTrain = {0:.2f}%, ".format(accuracyTrain), end = "")
print("buildTime = {0:.2f}s".format(buildingTime), end = "\n")

num_threads = nb.get_num_threads()
print("\n\nNumber of threads used by Numba:", num_threads)

AVG time to build a Decision Tree = 1.5082955360412598
  n = 1000, k = 10, maxDepth = 8:
    accTest = 69.31%, accTrain = 70.33%, buildTime = 16.26s


Number of threads used by Numba: 2


#### Second time run

In [8]:
startTime = time.time()
randomForest = createRandomForest(dataFrameTrain, bootstrapSize = 1000, forestSize = 10, treeMaxDepth = 8)
buildingTime = time.time() - startTime
randomForestTestResults = randomForestPredictions(dataFrameTest, randomForest)
accuracyTest = calculateAccuracy(randomForestTestResults, dataFrameTest.iloc[:, -1]) * 100
randomForestTrainResults = randomForestPredictions(dataFrameTrain, randomForest)
accuracyTrain = calculateAccuracy(randomForestTrainResults, dataFrameTrain.iloc[:, -1]) * 100
print("  n = {}, k = {}, maxDepth = {}:".format(1000, 10, 8))
print("    accTest = {0:.2f}%, ".format(accuracyTest), end = "")
print("accTrain = {0:.2f}%, ".format(accuracyTrain), end = "")
print("buildTime = {0:.2f}s".format(buildingTime), end = "\n")

num_threads = nb.get_num_threads()
print("\n\nNumber of threads used by Numba:", num_threads)

AVG time to build a Decision Tree = 0.1110929012298584
  n = 1000, k = 10, maxDepth = 8:
    accTest = 69.31%, accTrain = 70.33%, buildTime = 1.12s


Number of threads used by Numba: 2


### Run with breastCancer dataset

In [9]:
dataFrame = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/random-forest-from-scratch-main/dataset_files/breast_cancer.csv")
dataFrame = dataFrame.drop("id", axis = 1)
dataFrame = dataFrame[dataFrame.columns.tolist()[1: ] + dataFrame.columns.tolist()[0: 1]]
dataFrameTrain, dataFrameTest = trainTestSplit(dataFrame, testSize = 0.25)

print("Random Forest - Breast Cancer Dataset")
print("  Maximum bootstrap size (n) is {}".format(dataFrameTrain.shape[0]))
print("  Maximum random subspace size (d) is {}".format(dataFrameTrain.shape[1] - 1))

Random Forest - Breast Cancer Dataset
  Maximum bootstrap size (n) is 427
  Maximum random subspace size (d) is 30


#### First time run

In [10]:
startTime = time.time()
randomForest = createRandomForest(dataFrameTrain, bootstrapSize = 60, forestSize = 30, treeMaxDepth = 3)
buildingTime = time.time() - startTime
randomForestTestResults = randomForestPredictions(dataFrameTest, randomForest)
accuracyTest = calculateAccuracy(randomForestTestResults, dataFrameTest.iloc[:, -1]) * 100
randomForestTrainResults = randomForestPredictions(dataFrameTrain, randomForest)
accuracyTrain = calculateAccuracy(randomForestTrainResults, dataFrameTrain.iloc[:, -1]) * 100
print("  n = {}, k = {}, maxDepth = {}:".format(60, 30, 3))
print("    accTest = {0:.2f}%, ".format(accuracyTest), end = "")
print("accTrain = {0:.2f}%, ".format(accuracyTrain), end = "")
print("buildTime = {0:.2f}s".format(buildingTime), end = "\n")

AVG time to build a Decision Tree = 0.03280168374379476
  n = 60, k = 30, maxDepth = 3:
    accTest = 58.45%, accTrain = 64.17%, buildTime = 1.00s


#### Second time run

In [11]:
startTime = time.time()
randomForest = createRandomForest(dataFrameTrain, bootstrapSize = 60, forestSize = 30, treeMaxDepth = 3)
buildingTime = time.time() - startTime
randomForestTestResults = randomForestPredictions(dataFrameTest, randomForest)
accuracyTest = calculateAccuracy(randomForestTestResults, dataFrameTest.iloc[:, -1]) * 100
randomForestTrainResults = randomForestPredictions(dataFrameTrain, randomForest)
accuracyTrain = calculateAccuracy(randomForestTrainResults, dataFrameTrain.iloc[:, -1]) * 100
print("  n = {}, k = {}, maxDepth = {}:".format(60, 30, 3))
print("    accTest = {0:.2f}%, ".format(accuracyTest), end = "")
print("accTrain = {0:.2f}%, ".format(accuracyTrain), end = "")
print("buildTime = {0:.2f}s".format(buildingTime), end = "\n")

AVG time to build a Decision Tree = 0.04539896647135417
  n = 60, k = 30, maxDepth = 3:
    accTest = 58.45%, accTrain = 64.17%, buildTime = 1.38s
